#Assignment 6 : Mastering Big Data Handling
Group: COYS

|NAME|MATRICS NO.|
|---|---|
|MUHAMMAD HAZIM BIN SALMAN|A20EC0078|
|MUHAMMAD NAQUIB BIN ZAKARIA|A20BE0161|

##Loading the Dataset

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muhammadhazimas","key":"f565ee0e8bcb4dd61c17357a8b9a681f"}'}

In [2]:
pip install kaggle

In [3]:
mkdir ~/.kaggle

In [4]:
cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d cisautomotiveapi/large-car-dataset

 99% 818M/825M [00:11<00:00, 89.6MB/s]
100% 825M/825M [00:11<00:00, 74.2MB/s]


In [7]:
! unzip large-car-dataset.zip

Archive:  large-car-dataset.zip
  inflating: CIS_Automotive_Kaggle_Sample.csv  


##Strategies for Big Datasets

###1. Load Less Data

Import the pandas library for data manipulation, time for measuring time-related operations, and os for interacting with the operating system.

In [8]:
import pandas as pd
import time
import os

To calculate the file size of the dataset

In [9]:
file_path = "CIS_Automotive_Kaggle_Sample.csv"
file_size_mb = os.path.getsize(file_path) / (1024 ** 2)  # Convert to megabytes
file_size_gb = file_size_mb / 1024  # Convert to gigabytes

print("File size: {:.2f} GB".format(file_size_gb))

File size: 4.99 GB


Read data by specify which column to use

In [10]:
selected_columns = ['vin', 'askPrice', 'mileage', 'isNew', 'color', 'interiorColor','dealerID', 'brandName', 'modelName' ,'vf_BodyClass' ,'vf_FuelTypePrimary' , 'vf_ModelYear' , 'vf_PlantCountry','vf_VehicleType']
start_time = time.time()
df = pd.read_csv("CIS_Automotive_Kaggle_Sample.csv", usecols=selected_columns)
end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")

<ipython-input-10-90f700a5fd9c>:3: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("CIS_Automotive_Kaggle_Sample.csv", usecols=selected_columns)


Time taken: 81.21 seconds


Check the datatype for each column

In [ ]:
df.dtypes

vin                    object
askPrice                int64
mileage                 int64
isNew                    bool
color                  object
interiorColor          object
brandName              object
modelName              object
dealerID                int64
vf_BodyClass           object
vf_FuelTypePrimary     object
vf_ModelYear          float64
vf_PlantCountry        object
vf_VehicleType         object
dtype: object

Get the memory usage for the df dataframe

In [ ]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5695015 entries, 0 to 5695014
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   vin                 object 
 1   askPrice            int64  
 2   mileage             int64  
 3   isNew               bool   
 4   color               object 
 5   interiorColor       object 
 6   brandName           object 
 7   modelName           object 
 8   dealerID            int64  
 9   vf_BodyClass        object 
 10  vf_FuelTypePrimary  object 
 11  vf_ModelYear        float64
 12  vf_PlantCountry     object 
 13  vf_VehicleType      object 
dtypes: bool(1), float64(1), int64(3), object(9)
memory usage: 3.6 GB


count row and column

In [ ]:
df.shape

(5695015, 14)

Display first 5 row of the df dataframe

In [ ]:
df.head()

,vin,askPrice,mileage,isNew,color,interiorColor,brandName,modelName,dealerID,vf_BodyClass,vf_FuelTypePrimary,vf_ModelYear,vf_PlantCountry,vf_VehicleType
0,abc5f0360059cf7b6fa8368db57f220ab0d87d582bf6ce...,1498,0,False,Gray,NaN,MITSUBISHI,Eclipse Spyder,7514,Convertible/Cabriolet,NaN,2002.0,UNITED STATES (USA),PASSENGER CAR
1,e24402cc77f6fd2d6a9dd626ad92795b717814340f1ab1...,10589,0,False,Super Black,NaN,NISSAN,Altima,7514,Sedan/Saloon,Gasoline,2016.0,UNITED STATES (USA),PASSENGER CAR
2,1c5a8dc966b3d3b379e10477d9ff8a8aa13edf43acc7f7...,9940,0,False,White,NaN,FORD,Escape,7514,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,Gasoline,2014.0,UNITED STATES (USA),MULTIPURPOSE PASSENGER VEHICLE (MPV)
3,edce23814c88f5a1d1af700c27ad7f4c9d705aa9873317...,12387,0,False,Silver Ice Metallic,NaN,CHEVROLET,Cruze,7514,Sedan/Saloon,Gasoline,2017.0,UNITED STATES (USA),PASSENGER CAR
4,6b4f8c610d14d864f024adb26cd4f0eddcff3fa18c16e3...,41659,0,True,Black Metallic,NaN,FORD,F-150,7514,Pickup,Gasoline,2019.0,UNITED STATES (USA),TRUCK


###2. **Use Chunking**

In [ ]:
chunk_size = 100000
total_time = 0
total_memory = 0
start_time = time.time()
chunks = pd.read_csv(file_path, chunksize=chunk_size)
end_time = time.time()
print(f"Time taken: {end_time - start_time:.4f} seconds")

Time taken: 0.0147 seconds


Calculate Memory Usage and File Size

In [ ]:
total_rows = 0
total_columns = 0
for i, chunk in enumerate(chunks):

    # Get memory usage for the current chunk
    chunk_memory = chunk.memory_usage(deep=True).sum()

    # Accumulate total memory
    total_memory += chunk_memory

    total_rows += chunk.shape[0]
    total_columns = max(total_columns, chunk.shape[1])

# Calculate average time taken and memory usage per chunk
average_memory_per_chunk = total_memory / (i + 1) / (1024**3)
print(f"Average memory usage per chunk: {average_memory_per_chunk:.2f} GB")
print(f"\nOverall Shape of the Dataset:")
print(f"Total Rows: {total_rows}")
print(f"Total Columns: {total_columns}")

<ipython-input-19-53bf6c9927f1>:3: DtypeWarning: Columns (1,8,9,17,26,32,37,53,65,67,69,91,107,111,120,122) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
<ipython-input-19-53bf6c9927f1>:3: DtypeWarning: Columns (17,25,32,37,65,67,69,91,93,107,111,120,122) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
<ipython-input-19-53bf6c9927f1>:3: DtypeWarning: Columns (17,32,37,45,53,65,67,69,91,107,111,120,122) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
<ipython-input-19-53bf6c9927f1>:3: DtypeWarning: Columns (17,53,65,67,69,91,107,111,120,122) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
<ipython-input-19-53bf6c9927f1>:3: DtypeWarning: Columns (17,26,32,37,53,65,67,69,91,107,111,120,122) have mixed types. Specify dtype option on import 

Average memory usage per chunk: 0.50 GB

Overall Shape of the Dataset:
Total Rows: 5695015
Total Columns: 156


###3. **Optimize Data Types**

Copy Dataset to new dataframe


In [ ]:
df_optimized = df.copy()

To check null value in vf_ModelYear column

In [ ]:
df_optimized['vf_ModelYear'].isna().sum()

1282

To check unique value in vf_ModelYear column

In [ ]:
df_optimized['vf_ModelYear'].unique()

array([2002., 2016., 2014., 2017., 2019., 2018., 2013., 2012., 2007.,
       2009., 2010., 2005., 2008., 2015., 2001., 2006., 1999., 2003.,
       2011., 2004., 2000., 1997.,   nan, 1996., 2020., 1993., 1998.,
       1994., 1995., 1991., 1983., 1987., 1986., 1990., 2021., 1992.,
       1988., 1989., 1982., 1985., 1984., 1981., 1980.])

Replace null value with 0


In [ ]:
df_optimized['vf_ModelYear'] = df_optimized['vf_ModelYear'].fillna(0)

To check null value in vf_ModelYear column

In [ ]:
df_optimized['vf_ModelYear'].isna().sum()

0

Converting specific columns to lower precision data types

In [ ]:
numeric_to_convert = ['mileage', 'vf_ModelYear', 'dealerID']
df_optimized['askPrice'] = df_optimized['askPrice'].astype('float32')
df_optimized[numeric_to_convert] = df_optimized[numeric_to_convert].astype('int32')
object_to_convert = ['brandName', 'modelName', 'vf_BodyClass', 'vf_FuelTypePrimary', 'vf_PlantCountry', 'vf_VehicleType', 'color', 'interiorColor']
df_optimized[object_to_convert] = df_optimized[object_to_convert].astype('category')

Check the data types

In [ ]:
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5695015 entries, 0 to 5695014
Data columns (total 14 columns):
 #   Column              Dtype   
---  ------              -----   
 0   vin                 object  
 1   askPrice            float32 
 2   mileage             int32   
 3   isNew               bool    
 4   color               category
 5   interiorColor       category
 6   brandName           category
 7   modelName           category
 8   dealerID            int32   
 9   vf_BodyClass        category
 10  vf_FuelTypePrimary  category
 11  vf_ModelYear        int32   
 12  vf_PlantCountry     category
 13  vf_VehicleType      category
dtypes: bool(1), category(8), float32(1), int32(3), object(1)
memory usage: 813.2 MB


to display memory usage information for the entire DataFrame

In [ ]:
# Display the memory usage of the DataFrame
memory_usage_ori = df.memory_usage(deep=True)
memory_usage_opt = df_optimized.memory_usage(deep=True)

total_memory_usage_ori = memory_usage_ori.sum()
print(f"Total Memory Usage Before Optimize Data Type: {total_memory_usage_ori / (1024**3):.2f} GB")

total_memory_usage_opt = memory_usage_opt.sum()
print(f"Total Memory Usage After Optimize Data Type: {total_memory_usage_opt / (1024**3):.2f} GB")

total_memory_lost = (total_memory_usage_ori - total_memory_usage_opt)
print(f"Total Memory Reduction After Optimize Data Type: {total_memory_lost / (1024**3):.2f} GB")

percentage_reduction = ((total_memory_usage_ori - total_memory_usage_opt) / total_memory_usage_ori) * 100
print(f"Percentage Reduction in Memory Usage: {percentage_reduction:.2f}%")

Total Memory Usage Before Optimize Data Type: 3.62 GB
Total Memory Usage After Optimize Data Type: 0.79 GB
Total Memory Reduction After Optimize Data Type: 2.83 GB
Percentage Reduction in Memory Usage: 78.08%


###4. **Sampling**

Read a sample of a CSV file into a pandas DataFrame. The sample_size represents the number of rows that you want to read as a sample from the CSV file.

In [ ]:
sample_size = 1000000
start_time = time.time()
df_sample = pd.read_csv(file_path, nrows=sample_size)
end_time = time.time()

<ipython-input-28-b791e1ddaa5e>:3: DtypeWarning: Columns (1,8,9,17,22,25,26,32,37,53,65,67,69,70,91,93,107,111,120,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sample = pd.read_csv(file_path, nrows=sample_size)


Calculates the memory usage of the pandas DataFrame df_sample after reading a sample from a CSV file and then computes the total memory usage across all columns.

In [ ]:
memory_sample = df_sample.memory_usage(deep=True)
total_memory_sample = memory_sample.sum()

Display the time taken to read the sample DataFrame from the CSV file and the memory usage of the sample DataFrame

In [ ]:
print(f"Time taken: {end_time - start_time:.2f} seconds")
print(f"Memory Usage: {total_memory_sample / (1024**3):.2f} GB")

Time taken: 34.23 seconds
Memory Usage: 5.00 GB


###5. **Parallelize using Dask**

Installing the Dask library

In [ ]:
!pip install dask
import dask.dataframe as dd

Read a CSV file with specified data types for each column and Record the time taken to create the Dask DataFrame.

In [ ]:
start_time = time.time()
ddf = dd.read_csv(file_path, dtype={'interiorColor': 'object',
       'vf_AdaptiveHeadlights': 'object',
       'vf_AdditionalErrorText': 'object',
       'vf_AirBagLocSeatCushion': 'object',
       'vf_BatteryInfo': 'object',
       'vf_BatteryType': 'object',
       'vf_BrakeSystemDesc': 'object',
       'vf_ChargerLevel': 'object',
       'vf_CoolingType': 'object',
       'vf_DestinationMarket': 'object',
       'vf_DriverAssist': 'object',
       'vf_EDR': 'object',
       'vf_EVDriveUnit': 'object',
       'vf_LowerBeamHeadlampLightSource': 'object',
       'vf_ModelID': 'float64',
       'vf_NCSANote': 'object',
       'vf_OtherMotorcycleInfo': 'object',
       'vf_PossibleValues': 'object',
       'vf_Pretensioner': 'object',
       'vf_RearCrossTrafficAlert': 'object',
       'vf_Series2': 'object',
       'vf_SuggestedVIN': 'object',
       'vf_Trim': 'object',
       'vf_Trim2': 'object',
       'vf_WheelBaseType': 'object',
       'vf_AxleConfiguration': 'object',
       'vf_MakeID': 'float64',
       'vf_ManufacturerId': 'float64',
       'vf_ModelYear': 'float64',
       'vf_OtherTrailerInfo': 'object'})
end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")

Time taken: 0.04 seconds


display the ddf dataframe

In [ ]:
ddf.head()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


,vin,stockNum,firstSeen,lastSeen,msrp,askPrice,mileage,isNew,color,interiorColor,...,vf_VIN,vf_ValveTrainDesign,vf_VehicleType,vf_WheelBaseLong,vf_WheelBaseShort,vf_WheelBaseType,vf_WheelSizeFront,vf_WheelSizeRear,vf_Wheels,vf_Windows
0,abc5f0360059cf7b6fa8368db57f220ab0d87d582bf6ce...,11701A,2019-05-06,2019-05-06,1498,1498,0,False,Gray,NaN,...,abc5f0360059cf7b6fa8368db57f220ab0d87d582bf6ce...,NaN,PASSENGER CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e24402cc77f6fd2d6a9dd626ad92795b717814340f1ab1...,9055B,2019-05-06,2019-05-06,10589,10589,0,False,Super Black,NaN,...,e24402cc77f6fd2d6a9dd626ad92795b717814340f1ab1...,NaN,PASSENGER CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1c5a8dc966b3d3b379e10477d9ff8a8aa13edf43acc7f7...,11816A,2017-06-03,2019-05-07,11992,9940,0,False,White,NaN,...,1c5a8dc966b3d3b379e10477d9ff8a8aa13edf43acc7f7...,NaN,MULTIPURPOSE PASSENGER VEHICLE (MPV),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,edce23814c88f5a1d1af700c27ad7f4c9d705aa9873317...,11847,2019-05-06,2019-05-07,12387,12387,0,False,Silver Ice Metallic,NaN,...,edce23814c88f5a1d1af700c27ad7f4c9d705aa9873317...,Dual Overhead Cam (DOHC),PASSENGER CAR,NaN,106.3,NaN,17.0,17.0,4.0,4.0
4,6b4f8c610d14d864f024adb26cd4f0eddcff3fa18c16e3...,9097,2019-05-06,2019-05-07,41659,41659,0,True,Black Metallic,NaN,...,6b4f8c610d14d864f024adb26cd4f0eddcff3fa18c16e3...,NaN,TRUCK,156.8,145.0,NaN,17.0,17.0,4.0,NaN


Check datatypes for each column

In [ ]:
ddf.dtypes

vin                   object
stockNum              object
firstSeen             object
lastSeen              object
msrp                   int64
                      ...   
vf_WheelBaseType      object
vf_WheelSizeFront    float64
vf_WheelSizeRear     float64
vf_Wheels            float64
vf_Windows           float64
Length: 156, dtype: object

In [ ]:
ddf.info(memory_usage="deep")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (8,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (70,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio,

<class 'dask.dataframe.core.DataFrame'>
Columns: 156 entries, vin to vf_Windows
dtypes: object(98), bool(1), float64(53), int64(4)
memory usage: 6.6 GB


In [ ]:
num_partitions = ddf.npartitions

# Compute the length (number of rows) and width (number of columns) of each partition
partition_shapes = ddf.map_partitions(lambda df: df.shape, meta=(None, 'i8')).compute()

# Extract the number of rows and columns from a sample partition
num_rows, num_columns = partition_shapes.iloc[0]

print(f"Number of Partitions: {num_partitions}")
print(f"Number of Rows per Partition: {num_rows}")
print(f"Total of Rows: {num_rows * num_partitions}")
print(f"Number of Columns: {num_columns}")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (8,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (70,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio,

Number of Partitions: 83
Number of Rows per Partition: 69322
Total of Rows: 5753726
Number of Columns: 156


## Comparative Analysis

|Strategy|Computation Time|Memory Usage|File Size|
|-|-|-|-|
|Load Less Data|72.28 seconds|3.6 GB|Row : 5695015, Col : 14|
|Use Chunking|0.0033 seconds|0.50 GB|Row : 5695015, Col : 156|
|Optimize Data Types|-|0.79 GB|Row : 5695015, Col : 14|
|Sampling|44.84 seconds|5.00 GB|Row : 1000000, Col : 156|
|Parallelize with Dask|0.05 seconds|6.6 GB|Partition : 83, Row per Partition : 69322, Col : 156|

**Meaningful Insights**
* Load Less Data strategy took the longest time to compute, but it used the less amount of memory. This strategy is useful when you need to analyze a large dataset but only a small portion of it is relevant to your analysis.

* Use Chunking strategy was the fastest and used the least amount of memory. This strategy is useful when you need to process the data in smaller pieces to avoid memory issues.

* Optimize Data Types strategy used the least amount of memory, but we don’t have information on how long it took to compute. This strategy is useful when you need to fine-tune data types to maximize efficiency and minimize memory consumption.

* Sampling strategy took less time to compute than Load Less Data strategy, but it used more memory. This strategy is useful when you need to extract meaningful insights from a subset of the dataset.

* Parallelize with Dask strategy was the second fastest, but it used the most amount of memory. This strategy is useful when you need to parallelize computations across multiple cores or even multiple machines.

## Conclusion

In conclusion, handling large datasets effectively involves implementing a combination of smart strategies to optimize memory usage, processing efficiency, and computational resources.


| **Strategy** | **Description** |
|--------------|-----------------|
| Load Less Data | Strategically load only the essential portions of the dataset to optimize memory usage. This can be achieved by selecting specific columns or rows that are relevant to the analysis, or by loading the data in chunks. |
| Use Chunking | Process the data in smaller pieces to avoid memory issues. This can be done by dividing the data into smaller chunks and processing each chunk separately. |
| Optimize Data Types | Fine-tuning data types can maximize efficiency and minimize memory consumption. For example above we can se the reduction of memory usage is about 78.08%|
| Sampling | Implement sampling methodologies to extract meaningful insights from a subset of the dataset. This can be useful when the dataset is too large to process in its entirety. Sampling can be done randomly or systematically. |
| Parallelize with Dask | Dask is a powerful library that extends pandas to enable parallel and distributed computing. It's particularly useful for handling larger-than-memory datasets. Dask can be used to parallelize computations across multiple cores or even multiple machines. |

